In [1]:
from langchain.llms import GPT4All
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
import gradio as gr

In [2]:
model_weights = (r"C:\Users\chakumar\Downloads\wizardlm-13b-v1.1-superhot-8k.ggmlv3.q4_0.bin")

In [3]:
# initialize the LLM and make chain it with the prompts
Myllm = GPT4All(model = model_weights, backend="llama",)
#llm_chain = LLMChain(prompt=prompt, llm = Myllm, verbose = True)

Found model file at  C:\\Users\\chakumar\\Downloads\\wizardlm-13b-v1.1-superhot-8k.ggmlv3.q4_0.bin


In [4]:
doc_reader = PdfReader(r"C:\Users\chakumar\Documents\Documents\Intro_to_skill_prog.pdf")
raw_text = ''
for i, page in enumerate(doc_reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

In [5]:
# Splitting up the text into smaller chunks for indexing
text_splitter = CharacterTextSplitter(        
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap  = 200, #striding over the text
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [6]:
embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")
docsearch = FAISS.from_texts(texts, embeddings)

In [7]:
# set up FAISS as a generic retriever 
myretriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":4})

# create the chain to answer questions 
rqa_chain = RetrievalQA.from_chain_type(llm = Myllm, 
                                  chain_type = "stuff", 
                                  retriever = myretriever, 
                                  return_source_documents = True)

In [8]:
def inference(question):
    output = rqa_chain(question)
    return output['result']

In [18]:
def format_chat_prompt(query, chat_history):
    prompt = ""
    for turn in chat_history:
        user_query, bot_answer = turn
        prompt = f"{prompt}\nUser: {user_query}\nAssistant: {bot_answer}"
    prompt = f"{prompt}\nUser: {query}\nAssistant:"
    return prompt

def respond(query, chat_history):
    formated_query = format_chat_prompt(query, chat_history)
    answer = inference(formated_query)
    chat_history.append((query, answer))
    return "", chat_history
    
with gr.Blocks() as myApp:
    chatbot = gr.Chatbot(height = 240)
    question = gr.Textbox(label = "Question")
    ask_btn = gr.Button("Ask?")
    clear = gr.ClearButton(components = [question, chatbot], value = "Clear console")

    ask_btn.click(respond, inputs=[question, chatbot], outputs=[question, chatbot])
    question.submit(respond, inputs=[question, chatbot], outputs=[question, chatbot])
    
gr.close_all()
myApp.launch(share=True)

Closing server running on port: 7862
Closing server running on port: 7861
Closing server running on port: 7860
Running on local URL:  http://127.0.0.1:7861

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
